In [1]:
import numpy as np

## Utils

In [2]:
from sample import _generate_puzzle
import utils

def generate_puzzle(dimension, alpha, error_rate=0):
    w = utils.random_unit_vector(dimension)
    a, M1, _ = _generate_puzzle(w, dimension, alpha, error_rate=0)
    b, M2, coserror = _generate_puzzle(w, dimension, alpha, error_rate=error_rate)
    return a, b, M2 @ M1.T, coserror

## Example

In [6]:
import math
import numpy as np

dimension = 512
alpha = 4
# generate a random vector with dimension 36, which has only k non-zero elements(fill with 1 or -1) in random positions
# error_rate 0.15 -> 8.5 degrees; 0.25 -> 14 degrees; 0.35 -> 19 degrees; 
# 0.5 -> 26 degrees; 0.6 -> 31 degrees; 0.73 -> 36 degrees
error_rate = math.tan(70 * np.pi / 180)
a, b, isometric_matrix, coserror = generate_puzzle(dimension, alpha, error_rate=error_rate)     
# check if it is isometric
print(np.allclose(np.eye(dimension), np.dot(isometric_matrix, isometric_matrix.T)))
# check if it maps a to b
print(np.allclose(b, np.dot(isometric_matrix, a), atol=1e-3))
print("degree: ", np.arccos(np.clip(coserror, -1, 1)) / np.pi * 180)

print("Design angle:", np.arccos(1-1/alpha) / np.pi * 180/2)
print("M @ a:\t", np.sort(np.argsort(np.abs(isometric_matrix @ a))[-alpha:]))
print("b:\t", np.sort(np.argsort(np.abs(b))[-alpha:]))
print(np.allclose(np.sort(np.argsort(np.abs(isometric_matrix @ a))[-alpha:]), np.sort(np.argsort(np.abs(b))[-alpha:])))

True
False
degree:  69.99999999999999
Design angle: 20.70481105463543
M @ a:	 [ 22 189 258 269]
b:	 [ 22 189 269 367]
False


## Test for svd getting 2 sketches if sampled matrix is "correct"

In [15]:
import numpy as np
import utils
import math
from tqdm.auto import tqdm
from attacker import submatrix_solver_numpy
from ironmask import decode_codeword
# random select dimension // 2 positions that satisfy a[i] near 0
num_times = 0
success_times = 0
false_positive_times = 0
dimension = 512
tmpk = dimension // 2 + 220
print(utils.subset_n_a_alpha(dimension, tmpk, alpha) * 2)
# error_rate 0.15 -> 8.5 degrees; 0.25 -> 14 degrees; 0.35 -> 19 degrees; 
# 0.5 -> 26 degrees; 0.6 -> 31 degrees; 0.73 -> 36 degrees
error_rate = math.tan(75 * np.pi / 180)
threshold = 80


for j in (pbar_outer:=tqdm(range(10000))):
    dimension = 512
    alpha = 4
    # generate a random vector with dimension 512, which has only k non-zero elements(fill with 1 or -1) in random positions
    a, b, isometric_matrix, coserror = generate_puzzle(dimension, alpha, error_rate=error_rate)
    near_0_positions_a = np.where(np.abs(a) < 0.1)[0]
    near_0_positions_b = np.argsort(np.abs(b))[:dimension-alpha]
    # assert len(near_0_positions_a) == len(near_0_positions_b)
    # random select dimension // 2 near_0_positions
    test_number = 0
    coserror = 0
    min_angle = 90
    random_near_0_positions_a = np.random.choice(near_0_positions_a, tmpk, replace=False)
    random_near_0_positions_b = np.random.choice(near_0_positions_b, tmpk, replace=False)
    random_near_0_positions_a = np.sort(random_near_0_positions_a)
    random_near_0_positions_b = np.sort(random_near_0_positions_b)
    # get the submatrix of isometric_matrix with selected rows: random_near_0_positions_b and columns: versus random_near_0_positions_a
    versus_random_near_0_positions_a = np.setdiff1d(np.arange(dimension), random_near_0_positions_a)
    submatrix = isometric_matrix[random_near_0_positions_b][:, versus_random_near_0_positions_a]
    # get the null vector of submatrix
    null_vector = submatrix_solver_numpy(submatrix)
    null_vector = null_vector / np.linalg.norm(null_vector)
    assume_vector = np.zeros(dimension)
    for i in np.argsort(np.abs(null_vector)):
        assume_vector[versus_random_near_0_positions_a[i]] = null_vector[i]

    # determinant work
    assume_b = np.dot(isometric_matrix, assume_vector)
    decode_b = decode_codeword(assume_b, dimension, alpha)
    # get the arc cos of assume_b and b
    assume_a = np.dot(isometric_matrix.T, assume_b)
    decode_a = decode_codeword(assume_a, dimension, alpha)
    min_angle = utils.get_angle_of_two_vectors(assume_a, decode_a)
    if min_angle < threshold:  # pass determinant check
        if not np.allclose(np.sort(np.argsort(np.abs(decode_b))[-alpha:][::-1]), np.sort(np.argsort(np.abs(b))[-alpha:])):
            false_positive_times += 1
        else:
            success_times += 1
    pbar_outer.set_description("success_time: {}, false_positive_times: {}, whole_times: {}".format(success_times, false_positive_times, j+1))
    if success_times > 100:
        num_times = j + 1
        break

print(success_times / num_times)
print(success_times)

31.103979569752205


  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Test solve svd time

In [ ]:
import numpy as np
import attacker
import time

matrix_size = 512 + 400 - 1
matrix_size_2 = 512

tmp_num_times = 100
whole_time = 0
for i in tqdm(range(tmp_num_times)):
    tmp_matrix = np.random.rand(matrix_size, matrix_size_2)
    tmp_matrix = tmp_matrix[:-1]
    start_time = time.time()
    attacker.submatrix_solver_numpy(tmp_matrix)
    whole_time += time.time() - start_time

print(whole_time / tmp_num_times)
print("Whole Time: ", whole_time)

## Test run time for multiple matrices if sampled matrix is "correct"

In [ ]:
# reimport attacker
import importlib
import attacker
importlib.reload(attacker)

In [16]:
from attacker import solve_puzzle_with_n_matrix_known_places
from sample import generate_puzzle_n
from tqdm.auto import tqdm
import numpy as np
error_rate = math.tan(75 * np.pi / 180)
threshold = 80
dimension = 512
alpha = 4

c, bs, isometrixes, coserrors = generate_puzzle_n(dimension, alpha, error_rate = error_rate, n=1024, disable_tqdm=False)
# error parameter and angle degree relation
# for more information, see example.ipynb
# 0.035 -> 2.8 degrees; 0.07-> 5.6 degrees; 0.108 -> 8.6 degrees; 0.13 -> 11 degrees;  
# 0.18 -> 14 degrees; 0.25 -> 19 degrees; 0.35 -> 26 degrees; 0.4 -> 30 degrees; 0.5 -> 36 degrees; 

Generating Puzzles:   0%|          | 0/1024 [00:00<?, ?it/s]

In [17]:
total_run_times = 0
success_times = 0
false_positive_times = 0
total_test_times = 10000
tmpk = dimension - 1 + 400  # how many sketches do we choose
for j in (pbar:=tqdm(range(total_test_times), desc="Test for multiple matrices")):
    # random get (dimension - 1) bs and isometrixes
    indices = np.random.choice(len(bs), tmpk, replace=False).tolist()
    tmp_bs = [bs[i] for i in indices]
    tmp_isometrixes = [isometrixes[i] for i in indices]
    result = solve_puzzle_with_n_matrix_known_places(tmp_isometrixes, tmp_bs, dimension, alpha, max_rtimes=1, threshold=45, disable_tqdm=True, return_runtimes=True)
    if result[0] is not None:
        if (np.allclose(result[1], tmp_bs[0]) or np.allclose(result[1], -tmp_bs[0])):
            success_times += 1
        else:
            false_positive_times += 1
    total_run_times += result[2]
    pbar.set_postfix({"success_times": success_times, 
                      "averg_success_prob": success_times / total_run_times,
                      "false_positive_times": false_positive_times})
    if success_times > 100:
        num_times = j + 1
        break

print("Pr[Success]: ", success_times / num_times)
print("Pr[False postive]: ", false_positive_times / num_times)

Test for multiple matrices:   0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 